In [1]:
!python -V

Python 3.9.12


In [21]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2023/01/20 09:58:45 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/01/20 09:58:45 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='./mlruns/1', creation_time=1674208726586, experiment_id='1', last_update_time=1674208726586, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [8]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [10]:
len(df_train), len(df_val)

(73908, 61921)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.479618227604778

In [16]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [20]:
with mlflow.start_run():

    mlflow.set_tag("developer", "evert")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [18]:
lr = Lasso(0.1)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

12.143233939663894

In [22]:
import xgboost as xgb

In [23]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [26]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [24]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [27]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[12:12:47] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.                                    
[0]	validation-rmse:12.83410                                                                                                                              
[1]	validation-rmse:9.10000                                                                                                                               
[2]	validation-rmse:7.60632                                                                                                                               
[3]	validation-rmse:7.03168                                                                                                                               
[4]	validation-rmse:6.79239                                                                                                                               
[5]	validation-rmse:6.68910                                           

[51]	validation-rmse:6.51910                                                                                                                              
[52]	validation-rmse:6.51869                                                                                                                              
[53]	validation-rmse:6.51857                                                                                                                              
[54]	validation-rmse:6.51829                                                                                                                              
[55]	validation-rmse:6.51782                                                                                                                              
[56]	validation-rmse:6.51732                                                                                                                              
[57]	validation-rmse:6.51688                                          

[103]	validation-rmse:6.52101                                                                                                                             
[104]	validation-rmse:6.52138                                                                                                                             
[105]	validation-rmse:6.52176                                                                                                                             
[106]	validation-rmse:6.52173                                                                                                                             
[107]	validation-rmse:6.52153                                                                                                                             
[108]	validation-rmse:6.52172                                                                                                                             
[109]	validation-rmse:6.52231                                         

[34]	validation-rmse:6.52356                                                                                                                              
[35]	validation-rmse:6.52162                                                                                                                              
[36]	validation-rmse:6.52048                                                                                                                              
[37]	validation-rmse:6.51899                                                                                                                              
[38]	validation-rmse:6.51786                                                                                                                              
[39]	validation-rmse:6.51651                                                                                                                              
[40]	validation-rmse:6.51463                                          

[86]	validation-rmse:6.46834                                                                                                                              
[87]	validation-rmse:6.46809                                                                                                                              
[88]	validation-rmse:6.46747                                                                                                                              
[89]	validation-rmse:6.46620                                                                                                                              
[90]	validation-rmse:6.46603                                                                                                                              
[91]	validation-rmse:6.46565                                                                                                                              
[92]	validation-rmse:6.46467                                          

...

[582]	validation-rmse:6.35943                                                                                                                             
[583]	validation-rmse:6.35980                                                                                                                             
[584]	validation-rmse:6.35989                                                                                                                             
[585]	validation-rmse:6.35992                                                                                                                             
[586]	validation-rmse:6.35993                                                                                                                             
[587]	validation-rmse:6.35989                                                                                                                             
[588]	validation-rmse:6.35995                                         

[2]	validation-rmse:16.92663                                                                                                                              
[3]	validation-rmse:15.77377                                                                                                                              
[4]	validation-rmse:14.73486                                                                                                                              
[5]	validation-rmse:13.80275                                                                                                                              
[6]	validation-rmse:12.96519                                                                                                                              
[7]	validation-rmse:12.21631                                                                                                                              
[8]	validation-rmse:11.54784                                          

[54]	validation-rmse:6.51980                                                                                                                              
[55]	validation-rmse:6.51643                                                                                                                              
[56]	validation-rmse:6.51344                                                                                                                              
[57]	validation-rmse:6.51053                                                                                                                              
[58]	validation-rmse:6.50768                                                                                                                              
[59]	validation-rmse:6.50469                                                                                                                              
[60]	validation-rmse:6.50195                                          

[106]	validation-rmse:6.45783                                                                                                                             
[107]	validation-rmse:6.45725                                                                                                                             
[108]	validation-rmse:6.45663                                                                                                                             
[109]	validation-rmse:6.45624                                                                                                                             
[110]	validation-rmse:6.45580                                                                                                                             
[111]	validation-rmse:6.45535                                                                                                                             
[112]	validation-rmse:6.45514                                         

KeyboardInterrupt: 

In [28]:
# here put seperately & 'disable = True' to avoid logging the model twice
mlflow.xgboost.autolog(disable=True)

In [29]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[12:46:30] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95635
[2]	validation-rmse:16.59103
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29029
[5]	validation-rmse:13.32807
[6]	validation-rmse:12.47571
[7]	validation-rmse:11.72138
[8]	validation-rmse:11.05825
[9]	validation-rmse:10.47535
[10]	validation-rmse:9.96449
[11]	validation-rmse:9.51756
[12]	validation-rmse:9.12641
[13]	validation-rmse:8.78569
[14]	validation-rmse:8.49089
[15]	validation-rmse:8.23445
[16]	validation-rmse:8.01207
[17]	validation-rmse:7.81886
[18]	validation-rmse:7.65169
[19]	validation-rmse:7.50672
[20]	validation-rmse:7.38153
[21]	validation-rmse:7.27299
[22]	validation-rmse:7.17863
[23]	validation-rmse:7.09718
[24]	validation-rmse:7.02603
[25]	validation-rmse:6.96344
[26]	validation-rmse:6.90935
[27]	validation-rmse:6.86285
[28]	validation-rmse:6.82157
[29]	validation-rmse:6.78486
[30]	valida

[273]	validation-rmse:6.38760
[274]	validation-rmse:6.38752
[275]	validation-rmse:6.38711
[276]	validation-rmse:6.38693
[277]	validation-rmse:6.38679
[278]	validation-rmse:6.38631
[279]	validation-rmse:6.38589
[280]	validation-rmse:6.38573
[281]	validation-rmse:6.38517
[282]	validation-rmse:6.38490
[283]	validation-rmse:6.38479
[284]	validation-rmse:6.38458
[285]	validation-rmse:6.38413
[286]	validation-rmse:6.38405
[287]	validation-rmse:6.38383
[288]	validation-rmse:6.38357
[289]	validation-rmse:6.38302
[290]	validation-rmse:6.38281
[291]	validation-rmse:6.38251
[292]	validation-rmse:6.38235
[293]	validation-rmse:6.38202
[294]	validation-rmse:6.38163
[295]	validation-rmse:6.38126
[296]	validation-rmse:6.38063
[297]	validation-rmse:6.38038
[298]	validation-rmse:6.37990
[299]	validation-rmse:6.37973
[300]	validation-rmse:6.37959
[301]	validation-rmse:6.37926
[302]	validation-rmse:6.37923
[303]	validation-rmse:6.37911
[304]	validation-rmse:6.37905
[305]	validation-rmse:6.37882
[306]	vali

[547]	validation-rmse:6.33673
[548]	validation-rmse:6.33664
[549]	validation-rmse:6.33674
[550]	validation-rmse:6.33657
[551]	validation-rmse:6.33638
[552]	validation-rmse:6.33626
[553]	validation-rmse:6.33601
[554]	validation-rmse:6.33603
[555]	validation-rmse:6.33590
[556]	validation-rmse:6.33570
[557]	validation-rmse:6.33575
[558]	validation-rmse:6.33562
[559]	validation-rmse:6.33544
[560]	validation-rmse:6.33541
[561]	validation-rmse:6.33536
[562]	validation-rmse:6.33521
[563]	validation-rmse:6.33495
[564]	validation-rmse:6.33483
[565]	validation-rmse:6.33465
[566]	validation-rmse:6.33451
[567]	validation-rmse:6.33442
[568]	validation-rmse:6.33436
[569]	validation-rmse:6.33431
[570]	validation-rmse:6.33419
[571]	validation-rmse:6.33422
[572]	validation-rmse:6.33411
[573]	validation-rmse:6.33407
[574]	validation-rmse:6.33394
[575]	validation-rmse:6.33374
[576]	validation-rmse:6.33359
[577]	validation-rmse:6.33329
[578]	validation-rmse:6.33320
[579]	validation-rmse:6.33304
[580]	vali

[821]	validation-rmse:6.31206
[822]	validation-rmse:6.31200
[823]	validation-rmse:6.31205
[824]	validation-rmse:6.31194
[825]	validation-rmse:6.31188
[826]	validation-rmse:6.31180
[827]	validation-rmse:6.31174
[828]	validation-rmse:6.31150
[829]	validation-rmse:6.31147
[830]	validation-rmse:6.31113
[831]	validation-rmse:6.31115
[832]	validation-rmse:6.31116
[833]	validation-rmse:6.31122
[834]	validation-rmse:6.31111
[835]	validation-rmse:6.31107
[836]	validation-rmse:6.31105
[837]	validation-rmse:6.31100
[838]	validation-rmse:6.31083
[839]	validation-rmse:6.31074
[840]	validation-rmse:6.31056
[841]	validation-rmse:6.31045
[842]	validation-rmse:6.31045
[843]	validation-rmse:6.31039
[844]	validation-rmse:6.31026
[845]	validation-rmse:6.31024
[846]	validation-rmse:6.31018
[847]	validation-rmse:6.31003
[848]	validation-rmse:6.30992
[849]	validation-rmse:6.30993
[850]	validation-rmse:6.30999
[851]	validation-rmse:6.30970
[852]	validation-rmse:6.30967
[853]	validation-rmse:6.30965
[854]	vali

/home/evert_wydoodt/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [32]:
import mlflow
logged_model = 'runs:/df825c9f31854696bc95e58156af6e1e/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Load model as xgboost model;
xgboost_model = mlflow.xgboost.load_model(logged_model)

[14:50:35] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[14:50:35] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [33]:
xgboost_model

In [34]:
# Predict on a Pandas DataFrame.
y_pred = xgboost_model.predict(valid)

In [36]:
y_pred[:10]

array([14.954351,  7.142652, 14.259109, 24.31261 ,  9.302941, 17.112087,
       11.339904,  8.502365,  8.948904, 20.343412], dtype=float32)

In [37]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

KeyboardInterrupt: 